In [ ]:
from tqdm.contrib import tenumerate
from trajdl.datasets import Trajectory

all_trajs = [
    Trajectory(traj_pl.to_numpy(), entity_id=str(idx))
    for idx, traj_pl in tenumerate(original_trajs, desc="transform trajectorys")
]
print(all_trajs[0], all_trajs[0].seq[:10])